In [1]:
#import tools for web scraping:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

#group imports
import env
import acquire
import constants_prepare

import json
from typing import Dict, List, Optional, Union, cast
import requests

_____________________________________________________________________________

## Exploring Ubuntu:

In [2]:
ubuntu = pd.read_json('ubuntu_data.json')

In [3]:
ubuntu.head()

,repo,language,readme_contents
0,dockerfile/ubuntu,Shell,## Ubuntu Dockerfile\n\n\nThis repository cont...
1,boxcutter/ubuntu,Shell,# Packer templates for Ubuntu written in legac...
2,wszqkzqk/deepin-wine-ubuntu,C,# Deepin wine for Ubuntu and Debian\n\n## 一、项目...
3,fcwu/docker-ubuntu-vnc-desktop,HTML,# docker-ubuntu-vnc-desktop\n\n[![Docker Pulls...
4,docker-32bit/ubuntu,Shell,ubuntu\n======\n\nBuild a docker image for ubu...


#### Normalizing the data:

In [4]:
import unicodedata
import re

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

In [5]:
#to view just one of the readme docs and see what kind of normalizing is needed:
soup = BeautifulSoup(ubuntu.readme_contents.iloc[10], 'html.parser')
soup

Ubuntu 18.04 CIS STIG

[![Build Status](https://travis-ci.com/florianutz/Ubuntu1804-CIS.svg?branch=master)](https://travis-ci.com/florianutz/Ubuntu1804-CIS)
[![Ansible Role](https://img.shields.io/badge/role-florianutz.Ubuntu1804--CIS-blue.svg)](https://galaxy.ansible.com/florianutz/Ubuntu1804-CIS/)

Configure Ubuntu 18.04 machine to be CIS compliant. Level 1 and 2 findings will be corrected by default.

This role **will make changes to the system** that could break things. This is not an auditing tool but rather a remediation tool to be used after an audit has been conducted.

## IMPORTANT INSTALL STEP

If you want to install this via the `ansible-galaxy` command you'll need to run it like this:

`ansible-galaxy install -p roles -r requirements.yml`

With this in the file requirements.yml:

```
- src: https://github.com/florianutz/Ubuntu1804-CIS.git
```

Based on [CIS Ubuntu Benchmark v2.0.1 - 01-03-2020 ](https://www.cisecurity.org/cis-benchmarks/).

This repo originated from work do

Steps to parsing data:
- 1) Convert text to all lower case for normalcy.
- 2) Remove any accented characters, non-ASCII characters.
- 3) Remove special characters.
- 4) Stem or lemmatize the words.(stem = "if b, then c")
- 5) Remove stopwords.(if, and, the, etc)
- 6) Store the clean text and the original text for use in future notebooks.

In [7]:
ubuntu=constants_prepare.clean_data(ubuntu)

NameError: name 'nltk' is not defined

___________________________________

In [28]:
ubuntu_df = prep_article_data(ubuntu, 'readme_contents', ignore_columns = ['repo','language'], extra_words=[], exclude_words=[])

In [29]:
ubuntu_df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,dockerfile/ubuntu,Shell,## Ubuntu Dockerfile\n\n\nThis repository cont...,ubuntu dockerfile repository contains dockerfi...,ubuntu dockerfil repositori contain dockerfil ...,ubuntu dockerfile repository contains dockerfi...
1,boxcutter/ubuntu,Shell,# Packer templates for Ubuntu written in legac...,packer templates ubuntu written legacy json ov...,packer templat ubuntu written legaci json over...,packer template ubuntu written legacy json ove...
2,wszqkzqk/deepin-wine-ubuntu,C,# Deepin wine for Ubuntu and Debian\n\n## 一、项目...,deepin wine ubuntu debian deepinwine ubuntudeb...,deepin wine ubuntu debian deepinwin ubuntudebi...,deepin wine ubuntu debian deepinwine ubuntudeb...
3,fcwu/docker-ubuntu-vnc-desktop,HTML,# docker-ubuntu-vnc-desktop\n\n[![Docker Pulls...,dockerubuntuvncdesktop docker pullshttpsimgshi...,dockerubuntuvncdesktop docker pullshttpsimgshi...,dockerubuntuvncdesktop docker pullshttpsimgshi...
4,docker-32bit/ubuntu,Shell,ubuntu\n======\n\nBuild a docker image for ubu...,ubuntu build docker image ubuntu i386 run buil...,ubuntu build docker imag ubuntu i386 run build...,ubuntu build docker image ubuntu i386 run buil...


In [30]:
ubuntu_df.language.value_counts(normalize=True).head(10)

Shell         0.520747
Python        0.108921
Dockerfile    0.052905
C             0.045643
C++           0.035270
HTML          0.034232
Ruby          0.028008
JavaScript    0.024896
Jinja         0.018672
Go            0.012448
Name: language, dtype: float64

In [36]:
#creating df on languages:
shell_words = ubuntu_df.clean(' '.join(ubuntu_df[ubuntu_df.language == 'Shell']))
python_words = ubuntu_df.clean(' '.join(ubuntu_df[ubuntu_df.language == 'Python']))
all_words = ubuntu_df.clean(' '.join(ubuntu_df.str.text))


TypeError: 'Series' object is not callable